# CUDA and the GPU memory hierarchy

> Up to this point we have only seen and used the GPU global memory, which is the slowest memory from the GPU side. As we will see below there are several layers of memory that can be used during the execution of our kernels.

>![Alt text](images/memLayers.png)
>![Alt text](images/memCUDA2.png)


> Some specifications of these memories during the execution of the **kernel** are:


![Alt text](images/cudaMem.png)

It is important to note that each of these memories has different characteristics such as size, bandidth, latency, etc. The way to declare these memories within our kernel is the following:
![Alt text](images/defMem.png)
The access speeds are estimated below:
![Alt text](images/costMem.png)

> As you may have noticed, Texture memory has not been mentioned. This type of memory is linked to another CUDA structure called CUDA Arrays. These CUDA objects offer a structured way of generating arrays in memory (1D, 2D, 3D) , The disadvantage is that they are local memory spaces that we can not access directly inside our kernels. The reading of the information contained in CUDA Arrays is given only through the references known as Textures. CUDA Arrays can only be written by two methods, Surface Textures or copy between memory banks (Host or Device)


We will now see how to use the various memories, although the choice between which to use will depend exclusively on the problem to be solved. Let us simply compare each of the memories against the Global Memory by measuring the kernel's execution time.

In [ ]:
from pycuda import gpuarray
from pycuda.compiler import SourceModule
import numpy as np
import pycuda.driver as cuda
from pycuda.tools import DeviceData
from pycuda.tools import OccupancyRecord as occupancy

In [ ]:
%pylab inline

Part of what we have not seen, is to choose the GPU with which to work and generate a channel of communication (or context), we use the following function for that effect

In [ ]:
from CUDATools import *

In [ ]:
ctx

ctx,device = setDevice()
devdata = DeviceData(device)

def getKernelInfo(kernel,nthreads, rt=True):
    ''' This function returns info about kernels theoretical performance, but warning is not trivial to optimize! '''
    shared=kernel.shared_size_bytes
    regs=kernel.num_regs
    local=kernel.local_size_bytes
    const=kernel.const_size_bytes
    mbpt=kernel.max_threads_per_block
    #threads =  #self.block_size_x* self.block_size_y* self.block_size_z
    occupy = occupancy(devdata, nthreads, shared_mem=shared, registers=regs)
    print "==Kernel Memory=="
    print("""Local:        {0}
Shared:       {1}
Registers:    {2}
Const:        {3}
Max Threads/B:{4}""".format(local,shared,regs,const,mbpt))
    print "==Occupancy=="
    print("""Blocks executed by MP: {0}
Limited by:            {1}
Warps executed by MP:  {2}
Occupancy:             {3}""".format(occupy.tb_per_mp,occupy.limited_by,occupy.warps_per_mp,occupy.occupancy))
    if rt:
        return occupy.occupancy
    
def gpuMesureTime(myKernel, ntimes=1000):
    start = cuda.Event()
    end = cuda.Event()
    start.record()
    for i in range(ntimes):
      myKernel()
    end.record()
    end.synchronize()
    timeGPU = start.time_till(end)*1e-3
    print "Call the function {0} times takes in GPU {1} seconds.\n".format(ntimes,timeGPU)
    print "{0} seconds per call".format(timeGPU/ntimes)
    return timeGPU

If there is more than one GPU in the system (it is the case on Cartesius), the function is designed to ask which GPU to use

## Global memory vs. Registers vs. Shared memory vs. Constant memory.

> Below we outline a simple example of how to declare and use different types of memory within PyCUDA

In [ ]:
# Definimos los kernels 
Reg_Glob_RW = '''
__device__ __constant__ cuPres cMem=3.15149; // You can directly define constant memory 
                                                        
__global__ void rwRegisters(cuPres *A){

int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
int tid   = gridDim.x * blockDim.x * tid_y + tid_x;


cuPres pi = 3.141589; // Register memory write
A[tid] = pi; // Register lecture and Global Memory write

}

__global__ void rwGlobal(cuPres *A, cuPres *B){

int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
int tid   = gridDim.x * blockDim.x * tid_y + tid_x;

B[tid] = 3.141589cuStr; // Global memory write
A[tid] = B[tid]; // Global memory read and Global Memory write

}

__global__ void rwShared(cuPres *A){

int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
int tid   = gridDim.x * blockDim.x * tid_y + tid_x;
__shared__ cuPres pi; 
pi = 3.141589cuStr; // Shared memory write
//__syncthreads();
A[tid] = pi; // Shared memory read and Global Memory write
}

__global__ void rwSharedSync(cuPres *A){

int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
int tid   = gridDim.x * blockDim.x * tid_y + tid_x;
__shared__ cuPres pi; 
pi = 3.141589cuStr; // Shared memory write
__syncthreads();
A[tid] = pi; // Shared memory read and Global Memory write
}

__global__ void rwConstant(cuPres *A){

int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
int tid   = gridDim.x * blockDim.x * tid_y + tid_x;

A[tid] = cMem; // Constant memory read and Global Memory write
}

'''

## PRECISION

In [ ]:
#presCPU, presGPU = np.float32, 'float'
presCPU, presGPU = np.float64, 'double'

In [ ]:
def optKernels(kFile,pres='float',subBlGr = False, cuB=(1,1,1), cuG=(1,1,1)):
    kFile = kFile.replace('cuPres', pres)
    cString = 'f'
    if pres == 'double': cString = ''
    kFile = kFile.replace('cuStr', cString)
    if subBlGr: 
        downVar = ['blockDim.x','blockDim.y','blockDim.z','gridDim.x','gridDim.y','gridDim.z']
        upVar      = [str(cuB[0]),str(cuB[1]),str(cuB[2]),
                      str(cuG[0]),str(cuG[1]),str(cuG[2])]
        dicVarOptim = dict(zip(downVar,upVar))
        for i in downVar:
            kFile = kFile.replace(i,dicVarOptim[i])
    return kFile

In [ ]:
myKern = optKernels(Reg_Glob_RW,pres=presGPU,subBlGr=True)
print myKern

In [ ]:
cuCodeRG = SourceModule(myKern)
regRW = cuCodeRG.get_function("rwRegisters") 
gloRW = cuCodeRG.get_function("rwGlobal")
shaRW = cuCodeRG.get_function("rwShared")
shaSyRW = cuCodeRG.get_function("rwSharedSync")
conRW = cuCodeRG.get_function('rwConstant')
#conMemRW = cuCodeRG.get_global('cMem')[0] #We get a pointer to the constant memory address declared in the kernel
regRW.prepare('P')
gloRW.prepare('PP')
shaRW.prepare('P')
shaSyRW.prepare('P')
conRW.prepare('P')

>Note some differences between the use of each of the different memories, **Registers** do not present any difference to what we have been using so far, **Global** is not new, except that it is necessary to use two arrays in different global memory spaces. The **Shared** memory however requires an extra ingredient. As you will notice, there are two versions; The first one is asynchronous, while the second performs synchronization within the block. The line that changes between the two versions is *__syncthread()*. This command forces the kernel execution to wait for all threads within the block to have finished the tasks up to this point. The *advantage* of this function is that it allows to coordinate the execution of kernel tasks, while the *disadvantage* being a somewhat slower execution speed.

We now evaluate the performance of the various implementations

In [ ]:
from time import time

In [ ]:
nLoop = 1000
timeReg = []
timeGlo = []
timeSha = []
timeShaSy = []
timeCons = []
timeCPU = []
occK = []
cuBlock = (16,16,1) # 512,256,1024
threads = cuBlock[0]*cuBlock[1]*cuBlock[2]
cuGrid   = None
for i in range(8,14):
    
    
    N = 2**i
    a = np.ones((N,N),dtype=presCPU)
    b = np.ones((N,N),dtype=presCPU)
    A_gpu=gpuarray.to_gpu(a)
    B_gpu=gpuarray.to_gpu(b)
    #cuBlock = (16,16,1) # 512,256,1024
    threads = cuBlock[0]*cuBlock[1]*cuBlock[2]
    cuGrid   = (N/cuBlock[0],N/cuBlock[1],1)
    
    myKern  = optKernels(Reg_Glob_RW,pres=presGPU,subBlGr=True,cuB=cuBlock,cuG=cuGrid)
    cuCodeRG= SourceModule(myKern)
    regRW   = cuCodeRG.get_function("rwRegisters") 
    gloRW   = cuCodeRG.get_function("rwGlobal")
    shaRW   = cuCodeRG.get_function("rwShared")
    shaSyRW = cuCodeRG.get_function("rwSharedSync")
    conRW   = cuCodeRG.get_function('rwConstant')
    #conMemRW = cuCodeRG.get_global('cMem')[0] #Obtenemos un puntero a la direccion de memoria constante declarada en el kernel
    regRW.prepare('P')
    gloRW.prepare('PP')
    shaRW.prepare('P')
    shaSyRW.prepare('P')
    conRW.prepare('P')
    
    occK.append([getKernelInfo(regRW,nthreads=threads),getKernelInfo(gloRW,nthreads=threads),
                 getKernelInfo(shaRW,nthreads=threads),getKernelInfo(shaSyRW,nthreads=threads),
                 getKernelInfo(conRW,nthreads=threads)])
    
    t1 = 0
    t2 = 0
    t3 = 0
    t4 = 0
    t5 = 0
    tcpu = 0
    for k in range(nLoop):
        t_reg = regRW.prepared_timed_call(cuGrid,cuBlock,A_gpu.gpudata)
        #ctx.synchronize() 
        t_glo = gloRW.prepared_timed_call(cuGrid,cuBlock,A_gpu.gpudata,B_gpu.gpudata)
        #ctx.synchronize() 
        t_sha = shaRW.prepared_timed_call(cuGrid,cuBlock,A_gpu.gpudata)
        #ctx.synchronize() 
        t_shaSy = shaSyRW.prepared_timed_call(cuGrid,cuBlock,A_gpu.gpudata)
        #ctx.synchronize() 
        t_con = conRW.prepared_timed_call(cuGrid,cuBlock,A_gpu.gpudata)
        #ctx.synchronize()
        t = time()
        a[:,:] = np.pi
        b[:,:] = a
        t = time()-t
        t1 += t_reg()/nLoop
        t2 += t_glo()/nLoop
        t3 += t_sha()/nLoop
        t4 += t_shaSy()/nLoop
        t5 += t_con()/nLoop
        tcpu = t / nLoop
    timeReg.append(t1)
    timeGlo.append(t2)
    timeSha.append(t3)
    timeShaSy.append(t4)
    timeCons.append(t5)
    timeCPU.append(tcpu)
    A_gpu.gpudata.free()
    B_gpu.gpudata.free()

In [ ]:
print myKern

In [ ]:
occK,occK[:][0]

In [ ]:
plt.figure(1,figsize=(12,8),dpi=200)
plt.semilogx([2**i for i in range(8,14)],np.array(timeReg)*1e-3,'r-.*',label='Register',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],np.array(timeGlo)*1e-3,'b-*',label='Global',linewidth=3.0,alpha=0.9)
plt.semilogx([2**i for i in range(8,14)],np.array(timeSha)*1e-3,'g-*',label='Shared',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],np.array(timeShaSy)*1e-3,'m-*',label='Shared Sync',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],np.array(timeCons)*1e-3,'b--o',label='Constant',linewidth=3.0,alpha=0.5)
plt.ylabel('Time (sec)')
plt.xlabel('Matrix Dimension')
plt.xlim(xmin=250,xmax=2**13.1)
plt.title('Pure GPU')
plt.legend(loc=2,labelspacing=0.5,fancybox=True, handlelength=3.5, borderaxespad=0.25, borderpad=0.25)
plt.xticks([2**i for i in range(8,14)], [2**i for i in range(8,14)], rotation='vertical')

In [ ]:
plt.figure(1,figsize=(12,8),dpi=200)
plt.title('{0}, precision {1}, Block:{2}'.format(device.name(),presGPU,cuBlock),size=18)
plt.semilogx([2**i for i in range(8,14)],timeCPU,'.-',color=(0,0,0),label='CPU 1 thread',linewidth=2.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],1e-3*np.array(timeReg),'r-.*',label='Register',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],1e-3*np.array(timeGlo),'b-*',label='Global',linewidth=3.0,alpha=0.9)
plt.semilogx([2**i for i in range(8,14)],1e-3*np.array(timeSha),'g-*',label='Shared',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],1e-3*np.array(timeShaSy),'m-*',label='Shared Sync',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],1e-3*np.array(timeCons),'b--o',label='Constant',linewidth=3.0,alpha=0.5)
plt.ylabel('Time (sec)')
plt.xlabel('Matrix Dimension')
plt.xlim(xmin=250,xmax=2**13.1)
plt.legend(loc=2,labelspacing=0.5,fancybox=True, handlelength=3.5, borderaxespad=0.25, borderpad=0.25)
plt.xticks([2**i for i in range(8,14)], [2**i for i in range(8,14)], rotation='vertical')

In [ ]:
plt.figure(1,figsize=(12,8),dpi=200)
plt.title('{0}, precision {1}, Block:{2}'.format(device.name(),presGPU,cuBlock),size=18)
#plt.semilogx([2**i for i in range(8,14)],timeCPU,'.-',color=(0,0,0),label='CPU 1 thread',linewidth=2.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],np.array(timeCPU)/(1e-3*np.array(timeReg)),'r-.*',label='Register',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],np.array(timeCPU)/(1e-3*np.array(timeGlo)),'b-*',label='Global',linewidth=3.0,alpha=0.9)
plt.semilogx([2**i for i in range(8,14)],np.array(timeCPU)/(1e-3*np.array(timeSha)),'g-*',label='Shared',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],np.array(timeCPU)/(1e-3*np.array(timeShaSy)),'m-*',label='Shared Sync',linewidth=3.0,alpha=0.5)
plt.semilogx([2**i for i in range(8,14)],np.array(timeCPU)/(1e-3*np.array(timeCons)),'b--o',label='Constant',linewidth=3.0,alpha=0.5)
plt.ylabel('SpeedUp [times faster than CPU thread]')
plt.xlabel('Matrix Dimension')
plt.xlim(xmin=250,xmax=2**13.1)
plt.legend(loc=2,labelspacing=0.5,fancybox=True, handlelength=3.5, borderaxespad=0.25, borderpad=0.25)
plt.xticks([2**i for i in range(8,14)], [2**i for i in range(8,14)], rotation='vertical')